# Question 1 - Vitrebi Algorithm

### Stav Cohen 316492776
### Ron Kozitsa 312544240

### Background

This is an implementation of Vitrebi Algorithm with log probability:

Input:\
$S$ -> Set of possible states\
$O$ -> Sequence of observation $O_0, O_1, O_2... O_{T-1}$ \
$\sigma_v$ -> Start probability of state $S_v$ for each $S_v$ in $S$ \
$\omega_{u,v}$ -> Transition probability from $S_u$ to $S_v$ for each $S_u, S_v$ in $S$ \
$\alpha_v(O_t)$ -> Emission probability of observing $O_t$ from state $S_v$

Variables:\
$c_t(v)$ -> The max probability of any path ending in state $v$ at time $t$ \
$\pi_t(v)$ -> The state $u$ that maximizes the max probability of any path ending in state $v$ at time $t$

Output:\
${P_{T-1}}^*$ -> The best probability at the final observation \
${S_{T-1}}^*$ -> The state of the final observation that achieves the max probability \
$\forall t \in \{0, \ldots, T-2\} : {S_{t}}^*$ -> The path of states that achieves the max probability

Algorithm:

1. At the first observation $O_0$: \
$\forall{v\in{S}}: c_0(v) = -(\ln(\sigma_v) + \ln(\alpha_v(O_0)))$

2. For each observation $t$ from $O_1$ to $O_{T-1}$: \
$\forall{v\in{S}}: c_t(v) = min_{\forall{u\in{S}}}(-(\ln(c_{t-1}(u)) + \ln(\omega_{u,v}) + \ln(\alpha_v(O_t)))))$ \
$\forall{v\in{S}}: \pi_t(v) = argmin_{\forall{u\in{S}}}(-(\ln(c_{t-1}(u)) + \ln(\omega_{u,v}) + \ln(\alpha_v(O_t)))))$

3. When we finish the algorithm run: \
${P_{T-1}}^* = min_{\forall{v\in{S}}}(c_{T-1}(v))$ \
${S_{T-1}}^* = argmin_{\forall{v\in{S}}}(c_{T-1}(v))$

4. Now to find the best path we trace backward through the backpointer table from the final state: \
$\forall t \in \{0, \ldots, T-2\} : {S_t}^* = \pi_t({S_{t+1}}^*)$

In [10]:
import numpy as np
import pandas as pd

In [31]:
# Normalize probabilities -Ln
def to_log(p):
    return float(np.log(p)) * -1

## Vitrebi Algorithm

In [32]:
def viterbi(observations, states, start_probability, transition_probability, emission_probability):
    prob_matrix = []
    path = []

    prob_matrix.append([])

    # The probability of the first observation is based on the start probability and the emission probability of each one of the states
    for idx, y in enumerate(states):
        prob_matrix[0].append(to_log(start_probability[y]) + to_log(emission_probability[y][observations[0]]))
        path.append([idx])

    # For all the observations (From the second one)
    for t in range(1, len(observations)):
        prob_matrix.append([])
        newpath = []

        # Find the best probability and the best next state for each one of the pathes (Path for each state)
        # The best probability is based both on the prob of the previous observability, the transition probability
        # and the emission policy of each one of the states
        for idx, y in enumerate(states):
            probs = [prob_matrix[t-1][idx0] + to_log(transition_probability[y0][y]) + 
                     to_log(emission_probability[y][observations[t]]) for idx0, y0 in enumerate(states)]
            best_prob, best_state = float(np.min(probs)), np.argmin(probs)
            prob_matrix[t].append(best_prob)
            newpath.append(path[best_state] + [idx])

        path = newpath

    # Get the best probability and the best state of the final observation
    final_probs = [prob_matrix[-1][idx] for idx in range(len(states))]
    best_prob, best_state = float(np.min(final_probs)), np.argmin(final_probs)

    # Transform the best path from state indexes to state names
    best_path = [states[idx] for idx in path[best_state]]

    # Show the probability matrix as Dataframe
    log_probabilities_matrix = pd.DataFrame(prob_matrix)
    log_probabilities_matrix.rename_axis("Obs", axis=1, inplace=True)

    for idx, state in enumerate(states):
        log_probabilities_matrix.rename(columns={idx: f"Log Probability {state}"}, inplace=True)

    # Append the observation and the states of the best path to the DataFrame
    log_probabilities_matrix["Char"] = observations
    log_probabilities_matrix["State"] = best_path

    return (best_prob, best_path, log_probabilities_matrix)

## Vitrebi usage sample

In [33]:
states = ['L', 'R']
observations = "c c b a b d c a a c d b b".split()
start_probability = {'L': 0.5, 'R': 0.5}
transition_probability = {
    'L' : {'L': 0.55, 'R': 0.45},
    'R' : {'L': 0.4, 'R': 0.6},
}
emission_probability = {
   'L' : {'a': 0.15, 'b': 0.3, 'c': 0.35, 'd': 0.2},
   'R' : {'a': 0.35, 'b': 0.2, 'c': 0.15, 'd': 0.3},
}

In [34]:
prob, path, log_probabilities_matrix = viterbi(observations, states, start_probability, transition_probability, emission_probability)

## Results

In [35]:
print("Log Probabilities Matrix: \n")
display(log_probabilities_matrix)

Log Probabilities Matrix: 



Obs,Log Probability L,Log Probability R,Char,State
0,1.742969,2.590267,c,L
1,3.390628,4.438597,c,L
2,5.192438,5.798574,b,L
3,7.687395,7.040768,a,R
4,9.161032,9.161032,b,R
5,11.368307,10.875830,d,R
6,12.841943,13.283776,c,L
7,15.336900,14.690273,a,R
8,17.503683,16.250920,a,R
9,18.217033,18.658866,c,L


In [36]:
print("Log Probability: ", prob)

Log Probability:  24.027927825482564


In [37]:
print("Best Decoding Path: ", path)

Best Decoding Path:  ['L', 'L', 'L', 'R', 'R', 'R', 'L', 'R', 'R', 'L', 'L', 'L', 'L']
